# Проект SQL

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Наша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Наша первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

### Описание данных

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

### Задачи

- Посчитать, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитать количество обзоров и среднюю оценку;
- Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры;
- Определить автора с самой высокой средней оценкой книг — учитывать только книги с 50 и более оценками;
- Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [1]:
# импортируем библиотеки

import pandas as pd
from sqlalchemy import create_engine

# устанавливаем параметры

db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])

# сохраняем коннектор

engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

In [2]:
query = '''
        SELECT *
        FROM books
        '''

In [3]:
books = pd.io.sql.read_sql(query, con = engine)
pd.options.display.max_colwidth = 150
books.head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope #1),322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before Columbus,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
query2 = '''
        SELECT *
        FROM authors
        '''

In [5]:
authors = pd.io.sql.read_sql(query2, con = engine)
pd.options.display.max_colwidth = 150
authors.head()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [6]:
query3 = '''
        SELECT *
        FROM publishers
        '''

In [7]:
publishers = pd.io.sql.read_sql(query3, con = engine)
pd.options.display.max_colwidth = 150
publishers.head()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [8]:
query4 = '''
        SELECT *
        FROM ratings
        '''

In [9]:
ratings = pd.io.sql.read_sql(query4, con = engine)
pd.options.display.max_colwidth = 150
ratings.head()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [10]:
query5 = '''
        SELECT *
        FROM reviews
        '''

In [11]:
reviews = pd.io.sql.read_sql(query5, con = engine)
pd.options.display.max_colwidth = 150
reviews.head()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. Over provide race technology continue these.
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Among admit investment argue security.
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but person sport treatment industry. Kitchen decision deep the. Social party body the.
3,4,3,johnsonamanda,Finally month interesting blue could nature cultural bit. Prepare beat finish grow that smile teach. Dream me play near.
4,5,3,scotttamara,Nation purpose heavy give wait song will. List dinner another whole positive radio fast. Music staff many green.


### Задача 1

In [12]:
# Посчитаем, сколько книг вышло после 1 января 2000 года

query_1 = '''
        SELECT book_id, publication_date
        FROM books
        WHERE publication_date > '01-01-2000'
        '''

In [13]:
print('Количество книг:', pd.io.sql.read_sql(query_1, con = engine)['book_id'].count())
print('Крайняя дата публикации:', books['publication_date'].max())

Количество книг: 819
Крайняя дата публикации: 2020-03-31


<div class="alert alert-info">
    
- количество книг, опубликованных после 1 января 2000 г.: 819
- с 2000 года в среднем (по базе данных) публиковалось 40 книг в год
    
<div>

### Задача 2

In [14]:
# Для каждой книги посчитаем количество обзоров и среднюю оценку

query_2 = '''
        SELECT b.book_id,
               b.title,
               COUNT(r.review_id) AS count_reviews,
               AVG(ra.rating) AS avg_rating
        FROM books AS b
        LEFT JOIN authors AS a on b.author_id=a.author_id
        LEFT JOIN reviews AS r on b.book_id=r.book_id        
        LEFT JOIN ratings AS ra on b.book_id=ra.book_id
        GROUP BY b.book_id
        ORDER BY count_reviews DESC
        LIMIT 10;
        '''

In [15]:
display(pd.io.sql.read_sql(query_2, con = engine))

,book_id,title,count_reviews,avg_rating
0,948,Twilight (Twilight #1),1120,3.662500
1,750,The Hobbit or There and Back Again,528,4.125000
2,673,The Catcher in the Rye,516,3.825581
3,302,Harry Potter and the Prisoner of Azkaban (Harry Potter #3),492,4.414634
4,299,Harry Potter and the Chamber of Secrets (Harry Potter #2),480,4.287500
5,75,Angels & Demons (Robert Langdon #1),420,3.678571
6,301,Harry Potter and the Order of the Phoenix (Harry Potter #5),375,4.186667
7,779,The Lightning Thief (Percy Jackson and the Olympians #1),372,4.080645
8,722,The Fellowship of the Ring (The Lord of the Rings #1),370,4.391892
9,79,Animal Farm,370,3.729730


<div class="alert alert-info">
    
- наибольшее количество обзоров имеет книга Twilight (Сумерки.Сага.Часть 1)
- почти все книги из топ-10 были экранизированы, и имеют рекордные кассовые сборы в кинотеатрах
- 8 из них имеют несколько частей, являются франшизами, как в книгах, так и в экранизациях
- средний рейтинг топ-10 не ниже 3.5
    
<div>

### Задача 3

In [16]:
# Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры

query_3 = '''
        SELECT p.publisher_id,
               p.publisher,
               COUNT(b.book_id) AS count_books
               
        FROM publishers AS p
        LEFT JOIN books AS b on p.publisher_id=b.publisher_id
        WHERE b.num_pages > 50
        GROUP BY p.publisher_id
        ORDER BY count_books DESC
        LIMIT 10;
        '''

In [17]:
display(pd.io.sql.read_sql(query_3, con = engine))

,publisher_id,publisher,count_books
0,212,Penguin Books,42
1,309,Vintage,31
2,116,Grand Central Publishing,25
3,217,Penguin Classics,24
4,33,Ballantine Books,19
5,35,Bantam,19
6,45,Berkley,17
7,284,St. Martin's Press,14
8,46,Berkley Books,14
9,83,Delta,13


<div class="alert alert-info">
    
- наибольшее количество книг опубликовали Penguin Books - 42
- большая часть топа-10 являются основными издателями Amazon Books
- более 20 книг опубликовали лишь 4 издателя, это в среднем более 1 книги в год (по базе данных)
    
<div>

### Задача 4

In [18]:
# Определим автора с самой высокой средней оценкой книг — учтем только книги с 50 и более оценками

query_4 = '''
        SELECT a.author_id,
               a.author,
               AVG(r.rating) AS avg_rating,
               COUNT(r.rating) AS count_scores
                             
        FROM authors AS a
        LEFT JOIN books AS b on a.author_id=b.author_id
        LEFT JOIN ratings AS r on b.book_id=r.book_id
        GROUP BY a.author_id
        HAVING COUNT(r.rating) >= 50
        ORDER BY avg_rating DESC
        LIMIT 1;
        '''

In [19]:
display(pd.io.sql.read_sql(query_4, con = engine))

,author_id,author,avg_rating,count_scores
0,130,Diana Gabaldon,4.3,50


<div class="alert alert-info">
    
- Diana Gabaldon - автор с наибольшим средним рейтингом 4.3 (Максимальный - 5) из 50 оценок
- Наиболее известна серией любовно-фантастических романов «Чужестранка», первая книга серии вошла в 200 лучших книг по версии BBC. По мотивам романа «Чужестранка» телеканал Starz снял одноимённый телесериал.
    
<div>

### Задача 5

In [20]:
# Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

query_5 = '''
        with
        c_score AS (SELECT username
                    FROM ratings
                    GROUP BY username
                    HAVING COUNT(rating_id) > 50)
        ,
        c_review AS (SELECT username,
                            COUNT(review_id) AS count_reviews
                     FROM reviews
                     GROUP BY username)
        SELECT c_score.username AS user,
               AVG(c_review.count_reviews) AS avg_reviews
        FROM c_score
        LEFT JOIN c_review ON c_score.username=c_review.username
        GROUP BY c_score.username
        ORDER BY avg_reviews DESC
        LIMIT 10;               
        
        '''

In [21]:
display(pd.io.sql.read_sql(query_5, con = engine))

,user,avg_reviews
0,sfitzgerald,28.0
1,martinadam,27.0
2,richard89,26.0
3,jennifermiller,25.0
4,paul88,22.0
5,xdavis,18.0


<div class="alert alert-info">
    
- 6 читателей, сделавших более 50 оценок, оставляют в среднем от 18 до 28 обзоров
- в среднем эти читатели оставляют минимум от 40 до 55% обзоров, на общее количество прочитанных книг
- среди них вероятно нет русскоязычных читателей :)
    
<div>